In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="Ob9mLrO3fKqDwCHJu04j")
project = rf.workspace("surfline").project("surfer-spotting")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Surfer-Spotting-2 in yolov11:: 100%|██████████| 71100/71100 [00:05<00:00, 13332.94it/s]


In [9]:
import os
from ultralytics import YOLO
import yaml  # Still useful for potentially reading the yaml if needed later

# --- Configuration ---
# !! IMPORTANT: SET THIS PATH !!
# Path to your EXISTING data.yaml file
EXISTING_DATA_YAML_PATH = "/home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/data.yaml"
# Note: Ensure the paths (train, val, test) INSIDE your data.yaml are correct
# relative to the location of data.yaml itself. Your paths like '../train/images'
# suggest data.yaml should be directly inside Surfer-Spotting-2/.

# --- Output ---
# Where YOLOv8 will save training runs (logs, weights)
PROJECT_NAME = "YOLOv11_Surfer_Spotting"
# Specific name for this training experiment
RUN_NAME = "yolov11m_finetune_gpu"  # Updated run name slightly

# --- Training Parameters ---
MODEL_NAME = "yolo11m.pt"  # Pre-trained model to start from
EPOCHS = 1  # Adjust as needed
BATCH_SIZE = 16  # Adjust based on your GPU memory (may need to lower for smaller GPUs)
IMG_SIZE = 640  # Input image size
# Set device to '0' for the first CUDA GPU, or 'cuda' for default CUDA device
# Set to 'cpu' to force CPU training.
DEVICE = 0  # Or 'cuda', or specific GPU index like 1, 2 etc.

# --- 1. Verify data.yaml exists ---
if not os.path.exists(EXISTING_DATA_YAML_PATH):
    print(f"Error: Cannot find data.yaml at: {EXISTING_DATA_YAML_PATH}")
    print("Please ensure the path is correct.")
    exit()
else:
    print(f"Using existing data configuration: {EXISTING_DATA_YAML_PATH}")


Using existing data configuration: /home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/data.yaml


In [ ]:
print(f"Loading pre-trained model: {MODEL_NAME}")
model = YOLO(MODEL_NAME)


Loading pre-trained model: yolo11m.pt


In [ ]:


print(f"Starting training on device: {DEVICE}...")
try:
    results = model.train(
        data=EXISTING_DATA_YAML_PATH, 
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=RUN_NAME,
        device=DEVICE, 
    )
    print("Training finished.")
    print(f"Results saved to: {results.save_dir}")

except Exception as e:
    print(f"An error occurred during training: {e}")
    if "CUDA" in str(e):
        print(
            "A CUDA-related error occurred. Ensure CUDA is installed correctly, "
            "your NVIDIA drivers are up to date, and PyTorch with CUDA support is installed."
        )
        print(
            "You can check PyTorch CUDA availability with: python -c 'import torch; print(torch.cuda.is_available())'"
        )

#Eval
print(
    f"\nAttempting evaluation on the test set (if defined in YAML) using device: {DEVICE}..."
)
try:
   
    best_model_path = os.path.join(
        PROJECT_NAME, RUN_NAME, "weights", "best.pt"
    )
    if os.path.exists(best_model_path):
        print(f"Loading best weights from: {best_model_path}")
        model = YOLO(best_model_path)
        metrics = model.val(
            data=EXISTING_DATA_YAML_PATH, split="test", device=DEVICE
        )

        print("Test Set Evaluation Metrics:")

        print(f"  mAP50-95: {metrics.box.map:.4f}")
        print(f"  mAP50: {metrics.box.map50:.4f}")
        print(f"  mAP75: {metrics.box.map75:.4f}")
        
    else:
        print(
            f"Could not find best model weights at {best_model_path}. Skipping test set evaluation."
        )
except Exception as e:
    print(f"An error occurred during test set evaluation: {e}")
    print(
        "Ensure the 'test' key and path are correctly specified in your data.yaml if you want to evaluate."
    )


print("\nScript finished.")


Starting training on device: 0...
Ultralytics 8.3.135 🚀 Python-3.12.10 torch-2.7.0+cu126 
An error occurred during training: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.

A CUDA-related error occurred. Ensure CUDA is installed correctly, your NVIDIA drivers are up to date, and PyTorch with CUDA support is installed.
You can check PyTorch CUDA availability with: python -c 'import torch; print(torch.cuda.is_available())'

Attempting evaluation on the test set (if defined in YAML) using device: 0...
Could not find best model weights at YOLOv11_Surfer_Spotting/yolov11m_finetune_gpu/weights/best.pt. Skipping test set evaluation.

Script finished.
